In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D
from scipy.stats import linregress
from scipy.stats import ttest_rel, wilcoxon

In [ ]:
target_name = 'CD4 counts'

In [ ]:
metadf = pd.read_excel("/Users/alaa/Downloads/metadata_round1.xlsx")

In [ ]:
len(metadf.columns.tolist())

In [ ]:
metadf.columns

In [ ]:
metadf['PTID'].nunique()

In [ ]:
metadf.shape

In [ ]:
cd4df = metadf.loc[~metadf[target_name].isna()].copy()
print(cd4df.shape)

In [ ]:
cd4df['Time point'].unique()

In [ ]:
cd4df.loc[cd4df['group'] == "Defered", 'group'] = "Deferred"
cd4df.loc[cd4df['group'] == "Deferred ", 'group'] = "Deferred"
cd4df.loc[cd4df['group'] == "Immediate ", 'group'] = "Immediate"

In [ ]:
cd4df['group'].unique()

In [ ]:
# cd4df['Time point']

In [ ]:
df = cd4df.loc[cd4df['Time point'].isin(['DX', 'M12', 'M3','M6'])].copy()
df['timepoint'] = 0
df.loc[df['Time point']=='M3', 'timepoint'] = 1
df.loc[df['Time point']=='M6', 'timepoint'] = 2
df.loc[df['Time point']=='M12', 'timepoint'] = 3
df['timepoint'].value_counts()

In [ ]:
df['pid'] = df['PTID'].copy()

In [ ]:
df.loc[df['Viral loads']=='Undetectable', 'Viral loads'] = 0

In [ ]:
# df['cd4_counts'] = df['CD4 counts'].astype(float)
df[f"{target_name.lower().replace(' ', '_')}"] = df[target_name].astype(float)
# df['viral_load'] = df['Viral loads'].astype(int)

In [ ]:
if ignore_dx_timepoint:
    df = df.loc[df['timepoint']!=0].copy()
    df['timepoint'] -= 1

In [ ]:
plt.figure(figsize=(10, 6))

# Draw the boxplot for CD4 counts by timepoint
sns.boxplot(data=df, x='timepoint', y=f"{target_name.lower().replace(' ', '_')}", color='lightblue', showfliers=False)

# Define colors and markers for each group
unique_groups = df['group'].unique()
markers = ['o', '^']
colors = sns.color_palette("tab10", len(unique_groups))
group_styles = {
    group: {'color': colors[i], 'marker': markers[i % len(markers)]}
    for i, group in enumerate(unique_groups)
}

# Plot individual patient lines with dots styled by group
for pid, patient_df in df.groupby('pid'):
    patient_df = patient_df.sort_values('timepoint')
    group_val = patient_df['group'].iloc[0]
    style = group_styles[group_val]
    plt.plot(
        patient_df['timepoint'],
        patient_df[f"{target_name.lower().replace(' ', '_')}"],
        color=style['color'],
        marker=style['marker'],
        linestyle='-',
        alpha=0.7
    )

# Compute the mean CD4 count per group and timepoint
group_mean_df = df.groupby(['group', 'timepoint'], as_index=False)['cd4_counts'].mean()

# Compute paired differences for each group to get the slope and standard error.
# (Since there are only two timepoints, the slope is just the mean difference.)
slope_stats = {}
for group in unique_groups:
    group_df = df[df['group'] == group]
    # Pivot so that each row is a patient with columns for each timepoint
    pivot_df = group_df.pivot(index='pid', columns='timepoint', values=f"{target_name.lower().replace(' ', '_')}")
    # Compute the paired difference (timepoint 2 minus timepoint 1)
    paired_diff = pivot_df[2] - pivot_df[1]
    mean_diff = paired_diff.mean()  # This is the slope since (2-1)==1
    se_diff = paired_diff.std(ddof=1) / np.sqrt(len(paired_diff))
    slope_stats[group] = {'slope': mean_diff, 'se': se_diff}

# Plot the mean line for each group connecting the mean values across timepoints
for group in unique_groups:
    grp_mean = group_mean_df[group_mean_df['group'] == group].sort_values('timepoint')
    plt.plot(
        grp_mean['timepoint'],
        grp_mean[f"{target_name.lower().replace(' ', '_')}"],
        color=group_styles[group]['color'],
        marker='o',
        linestyle='--',
        linewidth=3,
        label=f"{group.capitalize()} Mean Line (slope = {slope_stats[group]['slope']:.2f})"
    )

plt.xlabel('Timepoint')
plt.ylabel(f'{target_name}')
plt.title(f'{target_name}')
plt.legend(title='Legend', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
# Create the boxplot for CD4 counts by timepoint
sns.boxplot(data=df, x='timepoint', y=f"{target_name.lower().replace(' ', '_')}", color='lightblue', showfliers=False)

# Automatically assign marker styles and colors for each unique group
unique_groups = df['group'].unique()
markers = ['o', '^']  # list of marker styles
colors = sns.color_palette("tab10", len(unique_groups))
group_styles = {
    group: {'color': colors[i], 'marker': markers[i % len(markers)]}
    for i, group in enumerate(unique_groups)
}

# Overlay lines and dots for each patient
for pid, group_df in df.groupby('pid'):
    group_df = group_df.sort_values('timepoint')  # ensure correct time order
    # Assume each patient belongs to a single group; get the first row's group
    group_val = group_df['group'].iloc[0]
    style = group_styles[group_val]
    # Plot the connecting line with markers
    plt.plot(
        group_df['timepoint'], 
        group_df[f"{target_name.lower().replace(' ', '_')}"], 
        color=style['color'], 
        marker=style['marker'], 
        linestyle='-', 
        alpha=0.7
    )
    
# Compute per-patient slopes and intercepts using linear regression
patient_params = []
for pid, patient_df in df.groupby('pid'):
    patient_df = patient_df.sort_values('timepoint')
    if len(patient_df) >= 2:
        res = linregress(patient_df['timepoint'], patient_df['cd4_counts'])
        group_val = patient_df['group'].iloc[0]
        patient_params.append({'pid': pid, 'group': group_val, 'slope': res.slope, 'intercept': res.intercept})
        
params_df = pd.DataFrame(patient_params)

# Compute group-level statistics for slopes: mean, standard deviation, count, and standard error
group_stats = params_df.groupby('group')['slope'].agg(['mean', 'std', 'count']).reset_index()
group_stats['se'] = group_stats['std'] / np.sqrt(group_stats['count'])

# Compute the mean intercept for each group (for plotting the mean line)
mean_intercepts = params_df.groupby('group')['intercept'].mean().reset_index()
group_stats = pd.merge(group_stats, mean_intercepts, on='group')  # columns: group, mean, std, count, se, intercept

# Determine the overall range of timepoints for plotting the mean lines
x_min = df['timepoint'].min()
x_max = df['timepoint'].max()
x_vals = np.linspace(x_min, x_max, 100)

# Plot the mean line for each group with slope and standard error annotated in the label
for idx, row in group_stats.iterrows():
    grp = row['group']
    mean_slope = row['mean']
    se = row['se']
    mean_intercept = row['intercept']
    y_vals = mean_intercept + mean_slope * x_vals
    plt.plot(
        x_vals, y_vals,
        color=group_styles[grp]['color'],
        linestyle='--',
        linewidth=3,
        label=f'{grp.capitalize()} Mean Line (slope = {mean_slope:.2f})'
    )

# To avoid duplicate legend entries from patients and mean lines, we deduplicate the legend
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), title='Legend', bbox_to_anchor=(0.05, 1), loc='upper left')

# Create a custom legend for the groups
# legend_elements = [
#     Line2D([0], [0], color=group_styles[group]['color'], marker=group_styles[group]['marker'], 
#            linestyle='-', label=group)
#     for group in group_styles
# ]
# plt.legend(handles=legend_elements, title="Group", bbox_to_anchor=(1.05, 1), loc='upper left')
# plt.ylim(0, 1000000)
plt.xlabel('Timepoint')
plt.ylabel(f'{target_name}')
plt.title(f'{target_name} per Patient Across Timepoints')
plt.tight_layout()
plt.show()

In [ ]:
# Ensure proper ordering
df = df.sort_values(['group', 'pid', 'timepoint'])

# Identify unique timepoints and groups
timepoints = sorted(df['timepoint'].unique())
groups = df['group'].unique()

results = []

# Loop over groups
for group in groups:
    group_df = df[df['group'] == group]
    # Pivot to have patients as rows and timepoints as columns
    pivot_df = group_df.pivot(index='pid', columns='timepoint', values=f"{target_name.lower().replace(' ', '_')}")
    
    # Loop over consecutive pairs of timepoints
    for i in range(len(timepoints) - 1):
        tp1 = timepoints[i]
        tp2 = timepoints[i+1]
        # Drop patients missing data at either timepoint
        paired_data = pivot_df[[tp1, tp2]].dropna()
        if len(paired_data) < 2:
            continue  # Skip if not enough pairs for a statistical test
        
        counts_tp1 = paired_data[tp1]
        counts_tp2 = paired_data[tp2]
        
        # Choose the paired t-test
        t_stat, p_value = ttest_rel(counts_tp1, counts_tp2)
        
        # Alternatively, you could use the Wilcoxon signed-rank test:
        # stat, p_value = wilcoxon(counts_tp1, counts_tp2)
        
        results.append({
            'group': group,
            'timepoint_pair': f"{tp1} vs {tp2}",
            'n': len(paired_data),
            't_stat': t_stat,
            'p_value': p_value
        })

results_df = pd.DataFrame(results)
print(results_df)